In [11]:
import matplotlib
matplotlib.use("Agg")

from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K
# from imutils import build_montages
import matplotlib.pyplot as plt
import numpy as np
# import cv2
# initialize the number of epochs to train for, base learning rate,
# and batch size
NUM_EPOCHS = 25
INIT_LR = 1e-2
BS = 32

In [13]:
# minivggnet build model

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K

In [15]:
class MiniVGGNet:
	@staticmethod
	def build(width, height, depth, classes):
		# initialize the model along with the input shape to be
		# "channels last" and the channels dimension itself
		model = Sequential()
		inputShape = (height, width, depth)
		chanDim = -1

		# if we are using "channels first", update the input shape
		# and channels dimension
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)
			chanDim = 1

		# first CONV => RELU => CONV => RELU => POOL layer set
		model.add(Conv2D(32, (3, 3), padding="same",
			input_shape=inputShape))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(32, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))

		# second CONV => RELU => CONV => RELU => POOL layer set
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))

		# first (and only) set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(512))
		model.add(Activation("relu"))
		model.add(BatchNormalization())
		model.add(Dropout(0.5))

		# softmax classifier
		model.add(Dense(classes))
		model.add(Activation("softmax"))

		# return the constructed network architecture
		return model

In [16]:
from tensorflow.keras.datasets import fashion_mnist
((trainX, trainY), (testX, testY)) = fashion_mnist.load_data()

In [17]:
print("[INFO] loading Fashion MNIST...")

[INFO] loading Fashion MNIST...


In [20]:
trainX.shape

(60000, 28, 28)

In [21]:
if K.image_data_format() == "channels_first":
	trainX = trainX.reshape((trainX.shape[0], 1, 28, 28))
	testX = testX.reshape((testX.shape[0], 1, 28, 28))
 
# otherwise, we are using "channels last" ordering, so the design
# matrix shape should be: num_samples x rows x columns x depth
else:
	trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
	testX = testX.reshape((testX.shape[0], 28, 28, 1))

In [22]:
trainX.shape

(60000, 28, 28, 1)

In [23]:
trainX = trainX.astype("float32") / 255.0
testX = testX.astype("float32") / 255.0
# one-hot encode the training and testing labels
trainY = to_categorical(trainY, 10)
testY = to_categorical(testY, 10)
# initialize the label names
labelNames = ["top", "trouser", "pullover", "dress", "coat","sandal", "shirt", "sneaker", "bag", "ankle boot"]

In [24]:
print("[INFO] compiling model...")
opt = SGD(lr=INIT_LR, momentum=0.9, decay=INIT_LR / NUM_EPOCHS)
model = MiniVGGNet.build(width=28, height=28, depth=1, classes=10)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
# train the network
print("[INFO] training model...")
H = model.fit(x=trainX, y=trainY,
	validation_data=(testX, testY),
	batch_size=BS, epochs=NUM_EPOCHS)

[INFO] compiling model...
[INFO] training model...
Epoch 1/25
1875/1875 [==============================] - 204s 109ms/step - loss: 0.5394 - accuracy: 0.8195 - val_loss: 0.3210 - val_accuracy: 0.8811
Epoch 2/25
1875/1875 [==============================] - 212s 113ms/step - loss: 0.3407 - accuracy: 0.8787 - val_loss: 0.2770 - val_accuracy: 0.8996
Epoch 3/25
1875/1875 [==============================] - 213s 114ms/step - loss: 0.2942 - accuracy: 0.8935 - val_loss: 0.2560 - val_accuracy: 0.9048
Epoch 4/25
1875/1875 [==============================] - 205s 109ms/step - loss: 0.2678 - accuracy: 0.9052 - val_loss: 0.2336 - val_accuracy: 0.9135
Epoch 5/25
1875/1875 [==============================] - 205s 109ms/step - loss: 0.2514 - accuracy: 0.9090 - val_loss: 0.2263 - val_accuracy: 0.9185
Epoch 6/25
1875/1875 [==============================] - 209s 111ms/step - loss: 0.2401 - accuracy: 0.9130 - val_loss: 0.2178 - val_accuracy: 0.9206
Epoch 7/25
1875/1875 [==============================] - 206s 

In [25]:
import cv2


ModuleNotFoundError: No module named 'cv2'

In [26]:

preds = model.predict(testX)

print("[INFO] evaluating network...")
print(classification_report(testY.argmax(axis=1), preds.argmax(axis=1),target_names=labelNames))

N = NUM_EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")

[INFO] evaluating network...
              precision    recall  f1-score   support

         top       0.90      0.88      0.89      1000
     trouser       0.99      0.99      0.99      1000
    pullover       0.89      0.92      0.90      1000
       dress       0.93      0.94      0.94      1000
        coat       0.88      0.91      0.90      1000
      sandal       0.99      0.99      0.99      1000
       shirt       0.81      0.78      0.79      1000
     sneaker       0.96      0.98      0.97      1000
         bag       0.99      0.99      0.99      1000
  ankle boot       0.98      0.96      0.97      1000

    accuracy                           0.93     10000
   macro avg       0.93      0.93      0.93     10000
weighted avg       0.93      0.93      0.93     10000



In [28]:
images = []

for i in np.random.choice(np.arange(0, len(testY)), size=(16,)):
	
	probs = model.predict(testX[np.newaxis, i])
	prediction = probs.argmax(axis=1)
	label = labelNames[prediction[0]]
 
	if K.image_data_format() == "channels_first":
		image = (testX[i][0] * 255).astype("uint8")

	else:
		image = (testX[i] * 255).astype("uint8")